In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
data = pd.read_csv('../traces/ray', header=None)
data.columns = ['branch_id', 'taken']
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42400131 entries, 0 to 42400130
Data columns (total 2 columns):
branch_id    int64
taken        int64
dtypes: int64(2)
memory usage: 647.0 MB


## 8192 Entry BHT (13-bit)

In [3]:
class BHT_core:
    def __init__(self, bits=13):
        self.bits = bits
        self.mask = 2**self.bits - 1
        self.table = np.zeros(2**self.bits, dtype=np.bool)
    
    def key(self, branch_id):
        return branch_id & self.mask
        
    def update(self, branch_id, value):
        self.table[self.key(branch_id)] = value
    
    def get(self, branch_id):
        return self.table[self.key(branch_id)]

In [4]:
class BHT_2_core:
    def __init__(self, bits=12):
        self.bits = bits
        self.mask = 2**self.bits - 1
        self.table = np.zeros(2**self.bits, dtype=np.uint8)
        self.states = {(0,0): 0, (0,1): 1,
                       (1,0): 0, (1,1): 3,
                       (2,0): 0, (2,1): 3,
                       (3,0): 2, (3,1): 3}
    
    def key(self, branch_id):
        return branch_id & self.mask
        
    def update(self, branch_id, value):
        self.table[self.key(branch_id)] = self.states[self.table[self.key(branch_id)], value]
    
    def get(self, branch_id):
        return self.table[self.key(branch_id)]//2

In [5]:
class BHT_2_2_core:
    def __init__(self, bits=10):
        self.bits = bits
        self.mask = 2**self.bits - 1
        self.table = np.zeros([2**self.bits, 4], dtype=np.uint8)
        self.states = {(0,0): 0, (0,1): 1,
                       (1,0): 0, (1,1): 3,
                       (2,0): 0, (2,1): 3,
                       (3,0): 2, (3,1): 3}
        self.global_history = np.zeros(1, dtype=np.uint8)
    
    def key(self, branch_id):
        return branch_id & self.mask
        
    def update(self, branch_id, value):
        self.table[self.key(branch_id), self.global_history[0]] = self.states[self.table[self.key(branch_id), self.global_history[0]], value]
        self.global_history[0] = (self.global_history << 1 | value) % 4
    
    def get(self, branch_id):
        return self.table[self.key(branch_id), self.global_history[0]]//2

In [6]:
class BHT_custom:
    def __init__(self, bits=10):
        self.bits = bits
        self.mask = 2**self.bits - 1
        self.table = np.zeros([2**self.bits, 4], dtype=np.uint8)
        self.states = {(0,0): 0, (0,1): 1,
                       (1,0): 0, (1,1): 2,
                       (2,0): 1, (2,1): 3,
                       (3,0): 2, (3,1): 3}
        self.history = np.zeros(2**self.bits, dtype=np.uint8)
    
    def key(self, branch_id):
        return branch_id & self.mask
        
    def update(self, branch_id, value):
        k = self.key(branch_id)
        self.table[k, self.history[k]] = self.states[self.table[k, self.history[k]], value]
        self.history[k] = (self.history[k] << 1 | value) % 4
    
    def get(self, branch_id):
        k = self.key(branch_id)
        return self.table[k, self.history[k]]//2

In [7]:
class BHT:
    def __init__(self, dataset, bht_type='BHT'):
        if bht_type.upper() == 'BHT':
            self.bht = BHT_core()
        elif bht_type.upper() == 'BHT_2':
            self.bht = BHT_2_core()
        elif bht_type.upper() == 'BHT_2_2':
            self.bht = BHT_2_2_core()
        elif bht_type.upper() == 'CUSTOM':
            self.bht = BHT_custom()
        self.codebase = dataset
    
    def run(self):
        bht = self.bht
        codebase = self.codebase
        total_predictions, mispredictions = (codebase.shape[0],0)
        start_time = time.clock()
        for i in range(codebase.shape[0]):
            branch_id = codebase.iloc[i].branch_id
            taken = codebase.iloc[i].taken
            if bht.get(branch_id) != taken:
                mispredictions = mispredictions + 1
            bht.update(branch_id,taken)
        end_time = time.clock()
        
        print('Total branches:        ', total_predictions)
        print('Unique branches:       ', codebase.branch_id.value_counts().shape[0])
        print('Correctly predicted:   ', total_predictions - mispredictions)
        print('Incorrectly predicted: ', mispredictions)
        print('Misprediction Rate:     %.2f' % (mispredictions/total_predictions*100))
        print('Total time elaped:      %.2f seconds' % (end_time-start_time))

## Custom

In [8]:
test = BHT(dataset=data, bht_type='custom')
test.run()

Total branches:         42400131
Unique branches:        797
Correctly predicted:    38692751
Incorrectly predicted:  3707380
Misprediction Rate:     8.74
Total time elaped:      11947.62 seconds


In [9]:
class BHT_custom_1:
    def __init__(self, bits=10):
        self.bits = bits
        self.mask = 2**self.bits - 1
        self.table = np.zeros([2**self.bits, 4], dtype=np.uint8)
        self.states = {(0,0): 0, (0,1): 1,
                       (1,0): 0, (1,1): 2,
                       (2,0): 1, (2,1): 3,
                       (3,0): 2, (3,1): 4,
                       (4,0): 3, (4,1): 5,
                       (5,0): 4, (5,1): 6,
                       (6,0): 5, (6,1): 7,
                       (7,0): 6, (7,1): 7}
        self.history = np.zeros(2**self.bits, dtype=np.uint8)
    
    def key(self, branch_id):
        return branch_id & self.mask
        
    def update(self, branch_id, value):
        k = self.key(branch_id)
        self.table[k, self.history[k]] = self.states[self.table[k, self.history[k]], value]
        self.history[k] = (self.history[k] << 1 | value) % 4
    
    def get(self, branch_id):
        k = self.key(branch_id)
        return self.table[k, self.history[k]]//4
    
bht = BHT_custom_1()
codebase = data
total_predictions, mispredictions = (codebase.shape[0],0)
start_time = time.clock()
for i in range(codebase.shape[0]):
    branch_id = codebase.iloc[i].branch_id
    taken = codebase.iloc[i].taken
    if bht.get(branch_id) != taken:
        mispredictions = mispredictions + 1
    bht.update(branch_id,taken)
end_time = time.clock()

print('Total branches:        ', total_predictions)
print('Unique branches:       ', codebase.branch_id.value_counts().shape[0])
print('Correctly predicted:   ', total_predictions - mispredictions)
print('Incorrectly predicted: ', mispredictions)
print('Misprediction Rate:     %.2f' % (mispredictions/total_predictions*100))
print('Total time elaped:      %.2f seconds' % (end_time-start_time))

Total branches:         42400131
Unique branches:        797
Correctly predicted:    38892319
Incorrectly predicted:  3507812
Misprediction Rate:     8.27
Total time elaped:      9815.00 seconds
